In [1]:
import gpflow
import numpy as np
import math
import tensorflow as tf
sess = tf.Session()

In [2]:
import get_history as gh

In [12]:
import feather
import pandas as pd

In [324]:



class CustomKernel(gpflow.kernels.Kernel):
    def __init__(self,input_dim,variance=1.0, lengthscales=1.0,
                 active_dims=None, ARD=None, name=None):
        super().__init__(input_dim)

        self.sigma_u = tf.constant(np.array([[100.0,10.0],[10.0,100.0]]))
        
        self.sigma_theta = tf.constant(np.array([[100.0,10.0,10.0],[10.0,100.0,10.0],\
                                    [10.0,10.0,100.0]]))
        #print(self.sigma_theta)
        #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  #dtype=gpflow.settings.float_type)
        
        self.sigma_v =  tf.constant(np.array([[100.0,0.0],[0.0,100.0]]))
        #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  #dtype=gpflow.settings.float_type)
        
        ##will it freak out if there is no parameter?
        
    
    @gpflow.params_as_tensors
    def rbf_custom(self, X, X2=None):
        if type(X2)==None:
            return tf.exp(-(X-X) / float(2.2))
        
        return tf.exp(-(X-X2) / float(2.2))
    
    @gpflow.params_as_tensors
    def _scaled_square_dist(self, X, X2):
        """
        Assumes each X has the form: [phi(state_action),psi(state_action),user_index,user_day_in_study_index]
        
        
        """
        X = X / self.lengthscales
        Xs = tf.reduce_sum(tf.square(X), axis=-1, keepdims=True)

        if X2 is None:
            dist = -2 * tf.matmul(X, X, transpose_b=True)
            dist += Xs + tf.matrix_transpose(Xs)
            return dist

        X2 = X2 / self.lengthscales
        X2s = tf.reduce_sum(tf.square(X2), axis=-1, keepdims=True)
        dist = -2 * tf.matmul(X, X2, transpose_b=True)
        dist += Xs + tf.matrix_transpose(X2s)
        return dist
    
    @staticmethod
    def _clipped_sqrt(r2):
        # Clipping around the (single) float precision which is ~1e-45.
        return tf.sqrt(tf.maximum(r2, 1e-40))
    
    def scaled_square_dist(self, X, X2):  # pragma: no cover
        return self._scaled_square_dist(X, X2)
    
    @gpflow.params_as_tensors
    def K_r2(self, X, X2):
        return self.variance * tf.exp(-r2 / 2.)
    
    @gpflow.params_as_tensors
    def K(self, X, X2=None):
        
        #feature_vector_one = X[0:4]
        #time_index_one = X[4:5]
        #user_index_one = X[5:6]
        
        
        #if X2!=None:
            #feature_vector_two = X2[0:4]
            #time_index_two = X2[4:5]
            #user_index_two = X2[5:6]

        user_id_one = X[-2]
        day_one = X[-1]
        f_one = X[0:3]
        g_one = X[3:5]
        rho_term = 1
        
        result = 1.0
        
        if not X2 is None:
            #print('called')
            
            user_id_two = X2[-2]
            day_two = X2[-1]
            f_two = X2[0:3]
            g_two = tf.transpose(X2[3:5])
        
        
            rho_term = self.rbf_custom(X[3:5],X2=X2[3:5]) 
            #print('rho')
            #print(rho_term)
            baselines = tf.tensordot(f_one,self.sigma_theta,axes=[[0],[1]])
            #print(baselines.shape)
            baselines = tf.tensordot(baselines,f_two,axes=[[0],[0]])
            #print('look at shape')
            #print(baselines)
            
            if user_id_one==user_id_two:
                user_term = self.sigma_u
            else:
                #print('user')
                #print('user_term')
                user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))
            #print(self.sigma_v)
            effect_term = tf.tensordot(rho_term,self.sigma_v,axes=[[0],[0]])
            effect_term = tf.add(user_term,effect_term)
            #print('eff')
            #print(effect_term.shape)
            #print('g')
            #print(g_one.shape)
            effects = tf.tensordot(g_one,effect_term,axes=[[0],[1]])
            #print('eff')
            #print(effects.shape)
            effects = tf.tensordot(effects,g_two,axes = [[0],[0]])
            resulf = tf.add(baselines,effects)
            #stuff to return
            
            
      
        
        return result
       
        
        #self.variance * tf.minimum(X, tf.transpose(X2))

    @gpflow.params_as_tensors
    def Kdiag(self, X, presliced=False):
        return tf.fill(tf.shape(X)[:-1], tf.squeeze(self.variance))

In [130]:
##need to reformat history
def put_x_together(h):
    X = []
    
    for i in range(len(h.states)):
        state_vector = h.states[i]
        state_vector.append(float(h.users[i]))
        state_vector.append(float(h.day_function[i]))
        X.append(state_vector)
    return X


In [128]:
history = gh.get_history(5,100,5)

In [131]:
X = put_x_together(history)

In [132]:
X = np.array(X)

In [141]:
y = np.array([[float(r)] for r in history.rewards])

In [319]:
k =  CustomKernel(5)
m = gpflow.models.GPR(X,y, kern=k)

Tensor("Const_293:0", shape=(3, 3), dtype=float64)


In [320]:
gpflow.train.ScipyOptimizer().minimize(m,session=sess)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 649.623368
  Number of iterations: 36
  Number of functions evaluations: 37


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 649.623368
  Number of iterations: 36
  Number of functions evaluations: 37


In [321]:
m.as_pandas_table()

,class,prior,transform,trainable,shape,fixed_shape,value
GPR/likelihood/variance,Parameter,None,+ve,True,(),True,25606.75161437026


In [330]:
def obtain_kernel_and_sigma_u(all_xs,all_ys):
    k =  CustomKernel(all_xs.shape[1])
    m = gpflow.models.GPR(all_xs, all_ys, kern=k)
    gpflow.train.ScipyOptimizer().minimize(m,session=sess)
    #sigma_u = m.kern.sigma_u
    #.value
    cov = np.zeros((all_xs.shape[0],all_xs.shape[0]))
    print(m.as_pandas_table())
    for i in range(all_xs.shape[0]):
        for j in range(all_xs.shape[0]):
            #print(type(all_xs[i][0]))
            term = m.kern.K(all_xs[i],all_xs[j])
            #term = tf.cast(term, tf.float32)
            cov[i][j]=term
            #term.eval(session=sess)
    return cov
#,sigma_u

In [331]:
cov,su = obtain_kernel_and_sigma_u(X,y)


Tensor("Const_5349:0", shape=(3, 3), dtype=float64)


KeyboardInterrupt: 

In [118]:
#one = tf.constant([0,0])
#two = tf.constant([1,1])
inp= tf.constant([[1.0, 2, 3], [4, 5, 6.0], [7.0, 8, 9]])
tf.norm([inp,inp],ord=2)


<tf.Tensor 'norm_9/Squeeze:0' shape=() dtype=float32>

In [13]:
df = pd.DataFrame(X,columns=['f_1','f_2','f_3','g_1','g_2','user_id','user_day_id'])
path = 'X_data.feather'
feather.write_dataframe(df, path)
df_rewards = pd.DataFrame(y,columns=['reward'])
path = 'y_data.feather'
feather.write_dataframe(df_rewards, path)

In [16]:

gr = m.kern.K(X,X)

In [19]:
X.shape

(2, 6)

In [20]:
gr.eval()

1.0